# Day 2 Homework – Ollama Webpage Summarizer

Upgrade the day 1 project to summarize a webpage using an **open-source model running locally via Ollama** instead of OpenAI.

**Benefits:** No API charges; data stays on your machine.  
**Trade-off:** Less capable than frontier models.

**Run from:** `week1/` or from this folder (`profe-ssor/`); the first code cell sets the path so `scraper` can be found.

## Step-by-step: Build the Ollama webpage summarizer

### Stage 1: Install Ollama (outside the notebook)

1. Go to **[ollama.com](https://ollama.com)** and install Ollama.
2. In a **Terminal**: `ollama serve`; in another: `ollama pull llama3.2` (or `ollama pull llama3.2:1b` if your machine is slow).
3. Open **http://localhost:11434/** in your browser — you should see **"Ollama is running"**.

In [ ]:
# Path setup + imports (run from week1 or from profe-ssor)
import os
import sys
from pathlib import Path

cwd = Path.cwd()
if cwd.name == "profe-ssor":
    week1_dir = cwd.parent.parent
elif cwd.name == "week1":
    week1_dir = cwd
else:
    week1_dir = cwd / "week1"
if str(week1_dir) not in sys.path:
    sys.path.insert(0, str(week1_dir))

from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI
import requests

In [ ]:
# Ollama client (local)
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

# Check Ollama is running
requests.get("http://localhost:11434").content

### Stage 2: Model name

Use `llama3.2` or `llama3.2:1b` (must match what you pulled).

In [ ]:
OLLAMA_MODEL = "llama3.2"   # or "llama3.2:1b" for slower machines

### Stage 3: Define prompts (same idea as day1)

System prompt = role/tone. User prompt = instruction + webpage text.

In [ ]:
system_prompt = """You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown. Do not wrap the markdown in a code block."""

user_prompt_prefix = """
Here are the contents of a website. Provide a short summary. If it includes news or announcements, summarize those too.

"""

### Stage 4: Build messages

The API expects a list: system + user (prefix + website content).

In [ ]:
def messages_for(website_text):
    """Build the messages list for the chat API: system prompt + user (prefix + website content)."""
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website_text}
    ]

### Stage 5: Summarize a URL with Ollama and display

Fetch the webpage, build messages, call Ollama, and show the result. Use a URL that works with the simple scraper (e.g. static HTML sites).

In [ ]:
# Ghana Education Service
url = "https://ges.gov.gh/"

# Fetch, build messages, call Ollama, display
website_text = fetch_website_contents(url)
messages = messages_for(website_text)
response = ollama.chat.completions.create(model=OLLAMA_MODEL, messages=messages)
display(Markdown(response.choices[0].message.content))